In [3]:
import pymanopt
import numpy as np

dim=3

In [9]:
man = pymanopt.manifolds.Sphere(dim)
A = np.random.rand(dim-1, dim-1)
cov = np.dot(A, A.transpose())
mean = man.random_point()

In [10]:
mean, cov

(array([-0.63195937, -0.73218917,  0.25402043]),
 array([[0.32529598, 0.65789412],
        [0.65789412, 1.82300413]]))

In [15]:
# generate tangent space
N = 100
samples = np.random.multivariate_normal(np.zeros(dim-1), cov, N)

In [26]:
# We need a tangent
normal = mean/np.linalg.norm(mean)

In [102]:
tol = 1e-5

def gram_schmidt(vectors):
    Q = np.zeros_like(vectors)
    for i in range(len(vectors)):
        v = vectors[i]
        for j in range(i):
            proj = np.dot(v, Q[j]) / np.dot(Q[j], Q[j]) * Q[j]
            v = v - proj
        Q[i] = v / np.linalg.norm(v)
    return Q
    
def get_axis(normal):
    # Assumes it is norm 1
    Q = np.eye(dim)
    arg = np.argmax(normal)
    if normal[arg]>(1-tol) and np.sum(normal)>(1-tol):
        return np.concatenate([Q[:arg], Q[arg+1:]])
    return gram_schmidt(np.vstack([normal,Q[:-1]]))[1:]

def get_e(i,n):
    return np.array(i*[0]+[1]+(n-i-1)*[0])

In [113]:
ax = get_axis(normal)

# Project points onto the axis to get all points in the tangent space
tangent_points = samples@ax

points = man.exp(mean, tangent_points)